In [1]:
import os
import json
import numpy as np
import nibabel as nib
from scipy.io import loadmat
from scipy.ndimage import binary_dilation

# =============================================================
# CONFIGURACIÓN
# =============================================================
BASE_DIR = "/media/guest/PORT-DISK/Practicas/MicroBleeds/IEEE-2016-N20/cmb-3dcnn-data"
IMAGES_DIR = os.path.join(BASE_DIR, "nii")
MASKS_DIR = os.path.join(BASE_DIR, "ground_truth")

# Dónde quieres almacenar el dataset nnUNet
NNUNET_RAW = "/media/guest/PORT-DISK/nnUNet_raw"
DATASET_ID = 2
DATASET_NAME = "Dou2016"

OUTPUT_DIR = os.path.join(NNUNET_RAW, f"Dataset{DATASET_ID:03d}_{DATASET_NAME}")
IMAGES_TR = os.path.join(OUTPUT_DIR, "imagesTr")
LABELS_TR = os.path.join(OUTPUT_DIR, "labelsTr")

os.makedirs(IMAGES_TR, exist_ok=True)
os.makedirs(LABELS_TR, exist_ok=True)


# =============================================================
# Función para crear máscara binaria a partir de coordenadas
# =============================================================
def create_mask_from_centers(shape, centers, radius=2):
    """
    Crea una máscara binaria 3D colocando esferas alrededor de cada CMB.
    """
    mask = np.zeros(shape, dtype=np.uint8)

    struct = np.zeros((2*radius+1, 2*radius+1, 2*radius+1), dtype=bool)
    rr, cc, zz = np.indices(struct.shape) - radius
    sphere = rr**2 + cc**2 + zz**2 <= radius**2

    for c in centers:
        x, y, z = map(int, c)
        if 0 <= x < shape[0] and 0 <= y < shape[1] and 0 <= z < shape[2]:
            mask[x, y, z] = 1
            # dilatar
            mask = binary_dilation(mask, structure=sphere)

    return mask.astype(np.uint8)


# =============================================================
# PROCESAMIENTO DE LOS 20 CASOS
# =============================================================
nii_files = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith(".nii")])

case_counter = 1

for nii_name in nii_files:
    case_id = f"{case_counter:03d}"

    img_path = os.path.join(IMAGES_DIR, nii_name)
    mat_path = os.path.join(MASKS_DIR, f"{nii_name.replace('.nii', '.mat')}")

    if not os.path.exists(mat_path):
        print(f"[AVISO] No existe {mat_path}, saltando caso.")
        continue

    # --- cargar imagen ---
    nii = nib.load(img_path)
    img = nii.get_fdata()
    affine = nii.affine

    # --- cargar centros ---
    mat = loadmat(mat_path)
    centers = mat["cen"]  # Nx3

    # --- generar máscara ---
    mask = create_mask_from_centers(img.shape, centers)

    # --- guardar imagen en nnUNet (convertida a .nii.gz) ---
    out_img = os.path.join(IMAGES_TR, f"{DATASET_NAME}_{case_id}_0000.nii.gz")
    nib.save(nib.Nifti1Image(img.astype(np.float32), affine), out_img)

    # --- guardar la máscara ---
    out_mask = os.path.join(LABELS_TR, f"{DATASET_NAME}_{case_id}.nii.gz")
    nib.save(nib.Nifti1Image(mask.astype(np.uint8), affine), out_mask)

    print(f"[OK] Caso {case_id} convertido.")
    case_counter += 1


# =============================================================
# GENERAR dataset.json
# =============================================================
dataset_json = {
    "name": DATASET_NAME,
    "description": "Dou 2016 Microbleeds dataset",
    "tensorImageSize": "3D",
    "reference": "Dou et al., 2016 IEEE TMI",
    "licence": "Unknown",
    "release": "1.0",
    "modality": {
        "0": "T2* / GRE"
    },
    "labels": {
        "0": "background",
        "1": "CMB"
    },
    "numTraining": 20,
    "training": [
        {
            "image": f"./imagesTr/{DATASET_NAME}_{i:03d}_0000.nii.gz",
            "label": f"./labelsTr/{DATASET_NAME}_{i:03d}.nii.gz"
        }
        for i in range(1, 21)
    ],
    "numTest": 0,
    "test": []
}

with open(os.path.join(OUTPUT_DIR, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=4)

print("\n==========================")
print(" CONVERSIÓN COMPLETADA ")
print("==========================")
print(f"Dataset nnUNet creado en:\n  {OUTPUT_DIR}")


[OK] Caso 001 convertido.
[OK] Caso 002 convertido.
[OK] Caso 003 convertido.
[OK] Caso 004 convertido.
[OK] Caso 005 convertido.
[OK] Caso 006 convertido.
[OK] Caso 007 convertido.
[OK] Caso 008 convertido.
[OK] Caso 009 convertido.
[OK] Caso 010 convertido.
[OK] Caso 011 convertido.
[OK] Caso 012 convertido.
[OK] Caso 013 convertido.
[OK] Caso 014 convertido.
[OK] Caso 015 convertido.
[OK] Caso 016 convertido.
[OK] Caso 017 convertido.
[OK] Caso 018 convertido.
[OK] Caso 019 convertido.
[OK] Caso 020 convertido.

 CONVERSIÓN COMPLETADA 
Dataset nnUNet creado en:
  /media/guest/PORT-DISK/nnUNet_raw/Dataset002_Dou2016
